In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
snapshot_dicts = dict()
rows_to_keep = []
snapshot_fork_revision_duplicated_removed = pd.DataFrame(columns=["snapshot_id", "fork_id", "date"])
row_count = 0
chunksize = 100000

for chunk in pd.read_csv('/mnt/17volume/data/snapshot_fork.csv.gz', compression='gzip', names=["date", "fork_id", "snapshot_id",], header=None, chunksize=chunksize):
    revision_point = chunk
    revision_point = revision_point.reset_index()
    revision_point = revision_point.sort_values(by=['date'], ascending=False)
    for i, row in revision_point.iterrows():
        snapshot_id = row['snapshot_id']
        fork_snapshot_id = row['fork_id']
        if snapshot_id not in snapshot_dicts:
            snapshot_dicts[snapshot_id] = dict()
        snapshot_dict = snapshot_dicts[snapshot_id]
        if fork_snapshot_id not in snapshot_dict:
            snapshot_dict[fork_snapshot_id] = (i + row_count, row['date'])
        else:
            if snapshot_dict[fork_snapshot_id][1] < row['date']:
                snapshot_dict[fork_snapshot_id] = (i + row_count, row['date'])
    row_count += chunksize

rows_to_keep = []
for i, snapshot_dict in snapshot_dicts.items():
    for j, (fork_snapshot_row_idx, date) in snapshot_dict.items():
        rows_to_keep.append(fork_snapshot_row_idx)
print("Number of forks to keep", len(rows_to_keep))
iter_csv = pd.read_csv('/mnt/17volume/data/snapshot_fork.csv.gz', compression='gzip', header=None, chunksize=chunksize)
df = pd.concat([chunk[chunk.index.isin(rows_to_keep)] for chunk in iter_csv])

KeyboardInterrupt: 

In [ ]:
df.head(5)

In [ ]:
df['year'] = df.apply(lambda row: datetime.utcfromtimestamp(row['date']).year, axis=1)
df['month'] = df.apply(lambda row: datetime.utcfromtimestamp(row['date']).month, axis=1)
df['day'] = df.apply(lambda row: datetime.utcfromtimestamp(row['date']).day, axis=1)

In [ ]:
df['year'].nunique()

In [ ]:
snapshot_fork_dict = dict()
for i, row in df.iterrows():
    if row.snapshot_id not in snapshot_fork_dict:
        snapshot_fork_dict[row.snapshot_id] = dict()
        
    if row.year not in snapshot_fork_dict[row.snapshot_id]:
        snapshot_fork_dict[row.snapshot_id][row.year] = [1, dict()]
    else:
        snapshot_fork_dict[row.snapshot_id][row.year][0] += 1
    
    if row.month not in snapshot_fork_dict[row.snapshot_id][row.year][1]:
        snapshot_fork_dict[row.snapshot_id][row.year][1][row.month] = 1
    else:
        snapshot_fork_dict[row.snapshot_id][row.year][1][row.month] += 1

print("The number of project with forks:", len(snapshot_fork_dict))
for snapshot, revision_date_set in snapshot_fork_dict.items():
    print(snapshot, len(revision_date_set))

In [ ]:
snapshot_ids = []
count_forks = []
count_years_exist = [] # the number of years between the first and last forks
count_months_exist = []
mean_forks = []
count_years_with_forks = []
count_months_with_forks = []
for snapshot, revision_date_set in snapshot_fork_dict.items():
#     print(snapshot)
    
    forks_year = []
    forks_month = []
    for year, (frequency, revision_date_set_month) in revision_date_set.items():
        forks_year.append((year, frequency))
        for month, freq in revision_date_set_month.items():
            forks_month.append((year, month, freq))
    forks_year.sort()
    forks_month.sort()
    count_fork = 0
    count_years_with_fork = 0
    count_months_with_fork = 0
    years = []
    months = []
    days = []
    for year, frequency in forks_year:
        years.append(year)
        count_fork += frequency
        count_years_with_fork += 1
#         print(year, frequency)
    for year, month, freq in forks_month:
        months.append((year, month))
        count_months_with_fork += 1
    months_exist = 0
    if months[-1][0] == months[0][0]:
        months_exist = months[-1][1] - months[0][1] + 1
    elif months[-1][1] < months[0][1]:
        extra_months = 12 * (months[-1][0] - months[0][0]) - 1
        months_exist = months[-1][1] + (12 - months[0][1] + 1) + extra_months
    else:
        extra_months = 12 * (months[-1][0] - months[0][0])
        months_exist = months[-1][1] - months[0][1] + extra_months
        
    snapshot_ids.append(snapshot)
    count_forks.append(count_fork)
    count_years_with_forks.append(count_years_with_fork)
    count_months_with_forks.append(count_months_with_fork)
    years_exist = years[-1] - years[0] + 1
    count_years_exist.append(years_exist)
    count_months_exist.append(months_exist)
    mean_forks.append(count_fork / years_exist)

df_fork_features = pd.DataFrame.from_dict({
    "snapshot_id": snapshot_ids,
    "count_forks": count_forks,
    "mean_forks": mean_forks,
    "count_years_with_forks": count_years_with_forks,
    "count_months_with_forks": count_months_with_forks,
    "count_years_exist": count_years_exist,
    "count_months_exist": count_months_exist
}, orient='index').transpose()

df_fork_features.head(16)

# number of forks
# mean number of forks by year
# number of year with at least one fork
# number of month with at least one fork
# number of days with at least one fork
# longest streak of years with at least one fork
# longest streak of months with at least one fork


In [ ]:
df_fork_features.to_csv("/home/sv/data/fork_pattern.csv")